# Topic Analysis of Review Data.

## DESCRIPTION

#### Help a leading mobile brand understand the voice of the customer by analyzing the reviews of their product on Amazon and the topics that customers are talking about. You will perform topic modeling on specific parts of speech. You’ll finally interpret the emerging topics.

## Problem Statement:

#### A popular mobile phone brand, Lenovo has launched their budget smartphone in the Indian market. The client wants to understand the VOC (voice of the customer) on the product. This will be useful to not just evaluate the current product, but to also get some direction for developing the product pipeline. The client is particularly interested in the different aspects that customers care about. Product reviews by customers on a leading e-commerce site should provide a good view.

#### Domain: Amazon reviews for a leading phone brand

##### Analysis to be done: POS tagging, topic modeling using LDA, and topic interpretation



##### Content: 
##### Dataset: ‘K8 Reviews v0.2.csv’
##### Columns:
##### Sentiment: The sentiment against the review (4,5 star reviews are positive, 1,2 are negative)
##### Reviews: The main text of the review

#### Steps to perform:
Discover the topics in the reviews and present it to business in a consumable format. Employ techniques in syntactic processing and topic modeling.
Perform specific cleanup, POS tagging, and restricting to relevant POS tags, then, perform topic modeling using LDA. Finally, give business-friendly names to the topics and make a table for business.

### Tasks: 

#### 1. Read the .csv file using Pandas. Take a look at the top few records.

In [1]:
import warnings
warnings.filterwarnings("ignore")

# Importing the usual utilities
import numpy as np,pandas as pd
import re,random,os,string

from pprint import pprint # pretty print
import matplotlib.pyplot as plt
%matplotlib inline

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from string import punctuation
from nltk.corpus import stopwords

In [2]:
reviews0=pd.read_csv('K8 Reviews v0.2.csv')
reviews0.head()

,sentiment,review
0,1,Good but need updates and improvements
1,0,"Worst mobile i have bought ever, Battery is dr..."
2,1,when I will get my 10% cash back.... its alrea...
3,1,Good
4,0,The worst phone everThey have changed the last...


#### 2. Normalize casings for the review text and extract the text into a list for easier manipulation.

In [3]:
reviews_lower=[sent.lower() for sent in reviews0.review.values]
reviews_lower[0]

'good but need updates and improvements'

#### 3. Tokenize the reviews using NLTKs word_tokenize function.

In [4]:
reviews_token=[word_tokenize(sent) for sent in reviews_lower]
reviews_token[0]

['good', 'but', 'need', 'updates', 'and', 'improvements']

#### 4. Perform parts-of-speech tagging on each sentence using the NLTK POS tagger.



In [5]:
nltk.pos_tag(reviews_token[0])

[('good', 'JJ'),
 ('but', 'CC'),
 ('need', 'VBP'),
 ('updates', 'NNS'),
 ('and', 'CC'),
 ('improvements', 'NNS')]

In [6]:
reviews_tagged=[nltk.pos_tag(tokens) for tokens in reviews_token]

In [7]:
reviews_tagged[0]

[('good', 'JJ'),
 ('but', 'CC'),
 ('need', 'VBP'),
 ('updates', 'NNS'),
 ('and', 'CC'),
 ('improvements', 'NNS')]

#### 5. For the topic model, we should  want to include only nouns.
 - Find out all the POS tags that correspond to nouns.
 - Limit the data to only terms with these tags.

In [8]:
nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

In [9]:
reviews_noun=[]
for sent in reviews_tagged:
    reviews_noun.append([token for token in sent if re.search("NN.*",token[1])])
reviews_noun[0]


[('updates', 'NNS'), ('improvements', 'NNS')]

In [10]:
reviews_noun[1]

[('mobile', 'NN'),
 ('i', 'NN'),
 ('battery', 'NN'),
 ('hell', 'NN'),
 ('backup', 'NN'),
 ('hours', 'NNS'),
 ('uses', 'NNS'),
 ('idle', 'NN'),
 ('discharged.this', 'NN'),
 ('lie', 'NN'),
 ('amazon', 'NN'),
 ('lenove', 'NN'),
 ('battery', 'NN'),
 ('charger', 'NN'),
 ('hours', 'NNS'),
 ('don', 'NN')]

In [11]:
pprint(reviews_noun[0][0][0])
pprint(reviews_noun[0][0][1])

'updates'
'NNS'


#### 6. Lemmatize. 
 - Different forms of the terms need to be treated as one.
 - No need to provide POS tag to lemmatizer for now.

In [12]:
lemm= WordNetLemmatizer()
reviews_lemm=[]   
for sent in reviews_noun:
    reviews_lemm.append([lemm.lemmatize(word[0]) for word in sent])

In [13]:
reviews_lemm[0]

['update', 'improvement']

#### 7. Remove stopwords and punctuation (if there are any). 

In [14]:
stop_nltk=stopwords.words("english")

In [15]:
stop_updated=stop_nltk + list(punctuation) + ["..."] + [".."]
reviews_sw_removed=[]
for sent in reviews_lemm:
    reviews_sw_removed.append([term for term in sent if term not in stop_updated])

In [16]:
reviews_sw_removed[1]

['mobile',
 'battery',
 'hell',
 'backup',
 'hour',
 'us',
 'idle',
 'discharged.this',
 'lie',
 'amazon',
 'lenove',
 'battery',
 'charger',
 'hour']

#### 8. Create a topic model using LDA on the cleaned up data with 12 topics.
 - Print out the top terms for each topic.
 - What is the coherence of the model with the c_v metric?

#### Use gensim

In [17]:
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.models import ldamodel

In [18]:
id2word = corpora.Dictionary(reviews_sw_removed)
texts = reviews_sw_removed
corpus = [id2word.doc2bow(text) for text in texts]

In [19]:
print(corpus[200])

[(36, 1), (143, 1), (314, 1), (415, 1), (416, 1)]


In [20]:
lda_model= gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=12,
                                           random_state=42,
                                           passes=10,
                                           per_word_topics=True)

In [21]:
pprint(lda_model.print_topics())

[(0,
  '0.138*"mobile" + 0.040*"call" + 0.036*"screen" + 0.031*"feature" + '
  '0.030*"option" + 0.020*"music" + 0.017*"software" + 0.016*"app" + '
  '0.015*"video" + 0.015*"card"'),
 (1,
  '0.151*"money" + 0.128*"...." + 0.071*"waste" + 0.056*"value" + '
  '0.046*"glass" + 0.038*"speaker" + 0.024*"gorilla" + 0.022*"set" + '
  '0.022*"ok" + 0.020*"piece"'),
 (2,
  '0.216*"note" + 0.113*"k8" + 0.090*"lenovo" + 0.030*"sound" + 0.023*"dolby" '
  '+ 0.020*"killer" + 0.018*"gallery" + 0.018*"system" + 0.018*"atmos" + '
  '0.018*"excellent"'),
 (3,
  '0.078*"phone" + 0.040*"day" + 0.038*"amazon" + 0.035*"service" + '
  '0.034*"issue" + 0.027*"time" + 0.027*"lenovo" + 0.026*"battery" + '
  '0.024*"month" + 0.023*"device"'),
 (4,
  '0.280*"product" + 0.176*"problem" + 0.080*"network" + 0.075*"issue" + '
  '0.066*"heating" + 0.021*"jio" + 0.021*"sim" + 0.019*"volta" + '
  '0.010*"connection" + 0.009*"signal"'),
 (5,
  '0.093*"heat" + 0.070*"....." + 0.052*"processor" + 0.038*"everything" + '
  

In [ ]:
# Compute coherence score
coherence_model_lda=CoherenceModel(model=lda_model,texts=reviews_sw_removed,dictionary=id2word,coherence='c_v')
coherence_lda=coherence_model_lda.get_coherence()
print('\nCoherence Score: ',coherence_lda)

#### Analyze the topics through the business lens.
 - Determine which of the topics can be combined.

##### If a pair of topics has very similar top terms, they are very close and can be combined

##### Looking at the topic and each terms the following can be combined - 
 - Topic 1,6,8 talks about pricing
 - Topic 9,10 mentions about phone performance
 - Topic 4,5 talsk about phone heating 

#### Create topic model using LDA with what you think is the optimal number of topics
- What is the coherence of the model?

In [ ]:
# Build LDA model

lda_model8 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=8,
                                            random_state=42,
                                            passes=10,
                                            per_word_topics=True))

In [ ]:
pprint(lda_model8.print_topics())

In [ ]:
# Compute coherence score
coherence_model_lda=CoherenceModel(model=lda_model8,texts=reviews_sw_removed,dictionary=id2word,coherence='c_v')
coherence_lda=coherence_model_lda.get_coherence()
print('\nCoherence Score: ',coherence_lda)

##### The business should  be able to interpret the topics.
 - Name each of the identified topics.
 - Create a table with the topic name and the top 10 terms in each to present to the  business.

In [ ]:
x=lda_model8.show_topics(formatted=False)
topic_words=[(tp[0],[wd[0] for wd in tp[1]]) for tp in x]

In [ ]:
for topic,words in topics_words:
    print(str(topic)+ "::" + str(words))
print()